In [6]:
import pandas as pd
import os

sym_dir = 'Symbols'
sym_files = [os.path.join(sym_dir, f) for f in os.listdir(sym_dir)]

In [451]:
class PinProperty:
    property_dict = {'PinName': '', 'PinNumber': '#=', 'PinType': 'PINTYPE='}
    idx = {'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 6, 'value': 7}
    just_dict = {1: 'Upper Left', 2: 'Middle Left', 3: 'Lower Left', 4: 'Upper Center', 5: 'Middle Center', 6: 'Lower Center', 7: 'Upper Right', 8: 'Middle Right', 9: 'Lower Right'}
    rotation_dict = {0: '0', 1: '90', 2: '180', 3: '270'}
    vis_dict = {0: 'Hidden', 2: 'Hidden-wProperty', 3: 'Visible', 4: 'Visible-wProperty'}
    
    def __init__(self, line_str, property_type):
        vals = line_str.split()
        p_typ = self.property_dict[property_type]
        
        self.x = int(vals[self.idx['x']])
        self.y = int(vals[self.idx['y']])
        self.size = int(vals[self.idx['size']])
        self.rotation = self.rotation_dict[int(vals[self.idx['rotation']])]
        self.justification = self.just_dict[int(vals[self.idx['justification']])]
        self.visible = self.vis_dict[int(vals[self.idx['visible']])]
        self.value = vals[self.idx['value']][len(p_typ):]

In [452]:
class PinType(PinProperty):
    def __init__(self, line_str):
        super().__init__(line_str, type(self).__name__)
        
class PinNumber(PinProperty):
    def __init__(self, line_str):
        super().__init__(line_str, type(self).__name__)

In [453]:
class PinName(PinProperty):
    idx = {'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 7, 'value': 9} # Line row values to string index
    vis_dict = {0: 'Hidden', 1: 'Visible'}
    
    def __init__(self, line_str):
        super().__init__(line_str, type(self).__name__)

In [454]:
# class PinName:
#     def __init__(self, line_str):
#         vals = line_str.split()
        
#         self.idx = {'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 7, 'name': 9} # Line row values to string index
#         self.just_dict = {1: 'Upper Left', 2: 'Middle Left', 3: 'Lower Left', 4: 'Upper Center', 5: 'Middle Center', 6: 'Lower Center', 7: 'Upper Right', 8: 'Middle Right', 9: 'Lower Right'}
#         self.rotation_dict = {0: '0', 1: '90', 2: '180', 3: '270'}
#         self.vis_dict = {0: 'Hidden', 1: 'Visible'}
        
#         self.x = self.str_to_val(vals, 'x')
#         self.y = self.str_to_val(vals, 'y')
#         self.size = self.str_to_val(vals, 'size')
#         self.rotation = self.str_to_val(vals, 'rotation', self.rotation_dict)
#         self.justification = self.str_to_val(vals, 'justification', self.just_dict)
#         self.visible = self.str_to_val(vals, 'visible', self.vis_dict)
#         self.value = self.str_to_val(vals, 'name')
        
#     def str_to_val(self, vals, col, idx_dict=None):
#         val = vals[self.idx[col]]
#         if idx_dict != None:
#             return idx_dict[int(val)]
#         elif all([v.isnumeric() for v in val]): # All characters are numeric
#             return int(val)
#         return val            

In [455]:
# class PinNumber:
#     def __init__(self, line_str):
#         vals = line_str.split()
        
#         self.idx = {'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 6, 'pin_number': 7}
#         self.just_dict = {1: 'Upper Left', 2: 'Middle Left', 3: 'Lower Left', 4: 'Upper Center', 5: 'Middle Center', 6: 'Lower Center', 7: 'Upper Right', 8: 'Middle Right', 9: 'Lower Right'}
#         self.rotation_dict = {0: '0', 1: '90', 2: '180', 3: '270'}
#         self.vis_dict = {0: 'Hidden', 2: 'Hidden-wProperty', 3: 'Visible', 4: 'Visible-wProperty'}
        
#         self.x = int(vals[self.idx['x']])
#         self.y = int(vals[self.idx['y']])
#         self.size = int(vals[self.idx['size']])
#         self.rotation = self.rotation_dict[int(vals[self.idx['rotation']])]
#         self.justification = self.just_dict[int(vals[self.idx['justification']])]
#         self.visible = self.vis_dict[int(vals[self.idx['visible']])]
#         self.value = vals[self.idx['pin_number']][len('#='):]

In [456]:
# class PinType:
#     def __init__(self, line_str):
#         vals = line_str.split()
        
#         self.idx = {'x': 1, 'y': 2, 'size': 3, 'rotation': 4, 'justification': 5, 'visible': 6, 'pin_type': 7}
#         self.just_dict = {1: 'Upper Left', 2: 'Middle Left', 3: 'Lower Left', 4: 'Upper Center', 5: 'Middle Center', 6: 'Lower Center', 7: 'Upper Right', 8: 'Middle Right', 9: 'Lower Right'}
#         self.rotation_dict = {0: '0', 1: '90', 2: '180', 3: '270'}
#         self.vis_dict = {0: 'Hidden', 2: 'Hidden-wProperty', 3: 'Visible', 4: 'Visible-wProperty'}
        
#         self.x = int(vals[self.idx['x']])
#         self.y = int(vals[self.idx['y']])
#         self.size = int(vals[self.idx['size']])
#         self.rotation = self.rotation_dict[int(vals[self.idx['rotation']])]
#         self.justification = self.just_dict[int(vals[self.idx['justification']])]
#         self.visible = self.vis_dict[int(vals[self.idx['visible']])]
#         self.value = vals[self.idx['pin_type']][len('PINTYPE='):]

In [467]:
class Pin:
    idx = {'id': 1, 'x1': 2, 'y1': 3, 'x2': 4, 'y2': 5, 'side': 7, 'inverted': 8} # Pin row to string index
    side_dict = {2: 'Left', 3: 'Right'}
    
    def __init__(self, line_str):
        vals = line_str.split()
        
        self.pid = vals[self.idx['id']]
        self.set_line_pos(vals[self.idx['x1']], vals[self.idx['x2']], vals[self.idx['y1']], vals[self.idx['y2']])
        self.side = self.side_dict[int(vals[self.idx['side']])]
        self.inverted = bool(int(vals[self.idx['inverted']]))
    
    # number: PinNumber Object
    def set_number(self, number):
        self.Number = number
    
    # ptype: PinType Object
    def set_type(self, ptype):
        self.Type = ptype
        
    def set_line_pos(self, x1, x2, y1, y2):
        self.line_pos = (int(x1), int(x2), int(y1), int(y2))
        
    # name: PinName Object
    def set_name(self, name):
        self.Name = name

In [468]:
class Symbol:
    def __init__(self):
        pass
    
    def set_name(self, name):
        self.name = name
        
    def set_date(self, date):
        self.date = date
        
    def units_to_mils(unit):
        return 100*unit/254000
    
    def parse_pin(self, pin_str_list):
        for line_str in pin_str_list:
            if line_str.startswith('P '):
                p = Pin(line_str)
            elif line_str.startswith('L '):
                p.set_name(PinName(line_str))
            elif '#=' in line_str:
                p.set_number(PinNumber(line_str))
            elif 'PINTYPE=' in line_str:
                p.set_type(PinType(line_str))
        return p

In [469]:
def get_symbol(file_path):
    sym = Symbol()
    NEWPIN = False
    pin_cnt = 0
    pin_list = []
    pin_str_list = []

    with open(file_path, 'r') as sym_file:
        for l in sym_file:
            if l.startswith('K '):
                sym.set_name(l.split()[-1])
            elif l.startswith('|R '):
                sym.set_date(l.split()[-1])
            elif l.startswith('P '):
                if pin_cnt > 0:
                    pin_list += [sym.parse_pin(pin_str_list)]
                NEWPIN = True
                pin_str_list = [l] # Reset List on each new pin
                pin_cnt += 1
            elif NEWPIN:
                pin_str_list += [l]
                if pin_cnt > 0:
                    pass
    return pin_list

In [470]:
pin_list = get_symbol(sym_files[0])

In [471]:
for p in pin_list:
    print('Name: {}, ID: {}, Number: {}, Type: {}, Side: {}, Inv: {}'.format(p.Name.value, p.pid, p.Number.value, p.Type.value, p.side, p.inverted))
    #print('Name: {}, Origin: {}, Side: {}, Rotation: {}'.format(p.name, p.value.txt_just, p.side, p.value.rotation))

Name: CLK_N, ID: 364, Number: 28, Type: IN, Side: Left, Inv: True
Name: CLK_P, ID: 488, Number: 27, Type: IN, Side: Left, Inv: False
Name: DB1_N, ID: 512, Number: 68, Type: OUT, Side: Right, Inv: True
Name: DB1_P, ID: 520, Number: 69, Type: OUT, Side: Right, Inv: False
Name: DB2_N, ID: 528, Number: 71, Type: OUT, Side: Right, Inv: True
Name: DB2_P, ID: 536, Number: 72, Type: OUT, Side: Right, Inv: False
Name: DB3_N, ID: 544, Number: 1, Type: OUT, Side: Right, Inv: True
Name: DB3_P, ID: 552, Number: 2, Type: OUT, Side: Right, Inv: False
Name: DA0_N, ID: 560, Number: 62, Type: OUT, Side: Right, Inv: True
Name: DA0_P, ID: 568, Number: 61, Type: OUT, Side: Right, Inv: False
Name: DA1_N, ID: 576, Number: 59, Type: OUT, Side: Right, Inv: True
Name: DA1_P, ID: 584, Number: 58, Type: OUT, Side: Right, Inv: False
Name: DA2_N, ID: 592, Number: 56, Type: OUT, Side: Right, Inv: True
Name: DA2_P, ID: 600, Number: 55, Type: OUT, Side: Right, Inv: False
Name: DA3_N, ID: 608, Number: 54, Type: OUT, Si